In [1]:
import pandas as pd
import requests
from datetime import datetime

# Deal with Injurty data

In [ ]:
df = pd.read_csv('injury.csv', header=0)

In [ ]:
df

In [ ]:
df['Date'] = df['Date'].apply(lambda x:'0'+str(x) if len(str(x)) == 1 else str(x))


In [ ]:
for n in df['Content'].unique():
    if 'rest' in n or 'returned' in n or 'sore' in n:
        print(n)

In [ ]:
delete = ['sore left foot', 'sore right knee','sore left shin','rest','general soreness / rest','returned to lineup','sore right shoulder']
for n in delete:
    df = df.drop(df.loc[df.Content == n].index)

In [ ]:
df.set_index('Date').to_csv('injury.csv',index='Date')

In [ ]:
df.drop_duplicates().set_index('Date').to_csv('injury.csv')

# Get data from NBA stats

In [ ]:
HEADERS = {'user-agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/45.0.2454.101 Safari/537.36'),
           'referer': 'http://stats.nba.com/scores/'
          }

In [ ]:
# Get all Kobe shot data from 1996 to 2016 and put it into an array
# This player ID comes from stats.nba.com (http://stats.nba.com/player/#!/977/stats/)
playerID = "977"
seasons = []
for season in range(1996,2016):
    # The stats.nba.com API wants season as "1996-97"
    seasonString = str(season) + '-' + str(season+1)[2:]

    # The stats.nba.com endpoint we are using is http://stats.nba.com/stats/shotchartdetail
    # More info on endpoints: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    shot_chart_url = 'http://stats.nba.com/stats/shotchartdetail?Period=0&VsConference=&LeagueID=00&LastNGames=0&TeamID=0&Position=&Location=&Outcome=&ContextMeasure=FGA&DateFrom=&StartPeriod=&DateTo=&OpponentTeamID=0&ContextFilter=&RangeType=&Season=' + seasonString + '&AheadBehind=&EndRange=&VsDivision=&PointDiff=&RookieYear=&GameSegment=&Month=0&ClutchTime=&StartRange=&EndPeriod=&SeasonType=Regular+Season&SeasonSegment=&GameID=&PlayerID=' + playerID + '&CFID=&PlayerPosition='
    response = requests.get(shot_chart_url, headers=HEADERS)
    # Split response into headers and content
    headers = response.json()['resultSets'][0]['headers']
    shots = response.json()['resultSets'][0]['rowSet']

    # Create pandas dataframe to hold the data
    shot_df = pd.DataFrame(shots, columns=headers)

    # add extra column for season
    shot_df['SEASON'] = seasonString

    # add extra column for playoff flag
    shot_df['playoffs'] = 0
    
    seasons.append(shot_df)
    

# Do the same thing for all the playoff shots
for season in range(1996,2016):
    seasonString = str(season) + '-' + str(season+1)[2:]

    # This URL is the same except for the parameter SeasonType=Playoffs
    shot_chart_url = 'http://stats.nba.com/stats/shotchartdetail?Period=0&VsConference=&LeagueID=00&LastNGames=0&TeamID=0&Position=&Location=&Outcome=&ContextMeasure=FGA&DateFrom=&StartPeriod=&DateTo=&OpponentTeamID=0&ContextFilter=&RangeType=&Season=' + seasonString + '&AheadBehind=&EndRange=&VsDivision=&PointDiff=&RookieYear=&GameSegment=&Month=0&ClutchTime=&StartRange=&EndPeriod=&SeasonType=Playoffs&SeasonSegment=&GameID=&PlayerID=' + playerID + '&CFID=&PlayerPosition='
    response = requests.get(shot_chart_url, headers=HEADERS)
    
    headers = response.json()['resultSets'][0]['headers']
    shots = response.json()['resultSets'][0]['rowSet']
    
    shot_df = pd.DataFrame(shots, columns=headers)
    shot_df['SEASON'] = str(season) + '-' + str(season+1)[2:]
    shot_df['playoffs'] = 1
    
    seasons.append(shot_df)

In [ ]:
# combine all season and playoffs dataframes into one dataframe
kobe_all_shots = pd.concat(seasons)
kobe_all_shots.columns = [n.lower() for n in kobe_all_shots.columns]
# dump a csv file
kobe_all_shots.to_csv("kobe_all_shots.csv")

# Add Score column for each shot

In [ ]:
df = pd.read_csv('kobe.csv', index_col=0)

In [ ]:
df['score'] = df['shot_made_flag'] * 1

In [ ]:
df['score'] = df['score'] * df['shot_type'].apply(lambda x:2 if '2' in x else 3)

In [ ]:
df.to_csv("kobe.csv")

# Add opponent column

In [2]:
data = pd.read_csv('kobe.csv', index_col=0)

In [3]:
data.shape

(30697, 17)

In [4]:
data['opponent'] = data['htm'].apply(lambda x: '' if x=='LAL' else x) + data['vtm'].apply(lambda x: '' if x=='LAL' else x)

In [5]:
data.to_csv("kobe.csv")